# Setup

In [1]:
!git clone https://github.com/IgnacioOQ/vaccine_ethics
%cd vaccine_ethics

Cloning into 'vaccine_ethics'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 66 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 23.66 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/vaccine_ethics


In [ ]:
from imports import *
from agent_class import FullAgent
from simulation_class import Simulation

# Testing

In [ ]:
simulation_results = pd.DataFrame(columns=["step", "dead_proportion", "max_infected", "auc_infected", "avg_viral_age", "avg_immunity"])

simulation = Simulation(grid_size=25, num_agents=600, agent_class = FullAgent, init_infected_proportion = 0.5,
                 proportion_vulnerable=0.1, vul_penalty = 0.5,
                 infection_prob=0.25, recovery_time=30, death_prob=0.05,
                 vax_vulnerable=False,
                 vax_all=False,
                 vax_effect = 0.7,
                 viral_age_effect = 0.1,
                 immune_adaptation_effect = 0.1,
                 plot=False)
simulation.run(500)  # Run for 20 iterations
# simulation.plot_hist()
results = simulation.generate_simulation_report()
simulation_results.loc[len(simulation_results)] = results
# simulation_results = pd.concat([simulation_results, results], ignore_index=True)
simulation_results

# More Bayesian Optimization with Parallelization

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.5 MB/s eta 0:00:00


In [ ]:
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from skopt import Optimizer
from skopt.space import Real
from skopt.plots import plot_convergence

# --- Simulation Constants (Scaled Up) ---
grid_size = 100
num_agents = 5000
init_infected_proportion = 0.1
proportion_vulnerable = 0.1
infection_prob = 0.25
recovery_time = 30

# --- Search Space ---
space = [
    Real(0.01, 0.1, name='death_prob'),
    Real(0.2, 0.8, name='vul_penalty'),
    Real(0.5, 0.9, name='vax_effect'),
    Real(0.05, 0.2, name='viral_age_effect'),
    Real(0.05, 0.2, name='immune_adaptation_effect')
]

# --- Objective Function ---

def run_simulation_for_params(params):
    """Run simulation for both vax_all True/False & return deaths difference."""
    death_prob, vul_penalty, vax_effect, viral_age_effect, immune_adaptation_effect = params
    deaths = {}

    for vax_all in [True, False]:
        simulation = Simulation(
            grid_size=grid_size,
            num_agents=num_agents,
            agent_class=FullAgent,
            init_infected_proportion=init_infected_proportion,
            proportion_vulnerable=proportion_vulnerable,
            vul_penalty=vul_penalty,
            infection_prob=infection_prob,
            recovery_time=recovery_time,
            death_prob=death_prob,
            vax_vulnerable=True,
            vax_all=vax_all,
            vax_effect=vax_effect,
            viral_age_effect=viral_age_effect,
            immune_adaptation_effect=immune_adaptation_effect,
            plot=False
        )
        simulation.run()
        report = simulation.generate_simulation_report()
        deaths[vax_all] = report[0]  # Assuming report[0] = dead count

    difference = deaths[True] - deaths[False]
    return difference

# --- Parallel Objective Function ---

def parallel_objective(params_batch, progress_desc="Running Batch", num_workers=None):
    """Parallel version with dynamic CPU worker usage."""
    if num_workers is None:
        num_workers = cpu_count()
    results = Parallel(n_jobs=num_workers)(
        delayed(run_simulation_for_params)(params) for params in tqdm(params_batch, desc=progress_desc)
    )
    return results

# --- Extract Near-Optimal Region ---

def get_best_region(df_results, threshold_percent=10):
    """Filter parameter sets within threshold % of the best result."""
    best_val = df_results['deaths_diff (vax_all True - False)'].min()
    threshold = abs(best_val) * (threshold_percent / 100)
    near_best = df_results[
        abs(df_results['deaths_diff (vax_all True - False)'] - best_val) <= threshold
    ]
    print(f"\nFound {len(near_best)} parameter sets within {threshold_percent}% of the best result.")
    near_best.to_csv('near_best_parameter_region.csv', index=False)
    print("\nSaved near-optimal parameter sets to 'near_best_parameter_region.csv'.")
    print(near_best[['death_prob', 'vul_penalty', 'vax_effect', 'viral_age_effect', 'immune_adaptation_effect']])
    return near_best

# --- Bayesian Optimization Pipeline ---

def run_bayesian_optimization(
    n_calls=500,
    n_initial_points=50,
    parallel_batch_size=50,
    threshold_percent=10,
    num_workers=None
):
    print("\n--- Starting Bayesian Optimization ---")
    results_list = []

    def batch_objective(params_batch, desc):
        batch_results = parallel_objective(params_batch, progress_desc=desc, num_workers=num_workers)
        # Record results
        for p, res in zip(params_batch, batch_results):
            results_list.append({
                'death_prob': p[0],
                'vul_penalty': p[1],
                'vax_effect': p[2],
                'viral_age_effect': p[3],
                'immune_adaptation_effect': p[4],
                'deaths_diff (vax_all True - False)': res
            })
        return batch_results

    # Initialize optimizer
    opt = Optimizer(dimensions=space, base_estimator="GP", acq_func="EI", random_state=42)

    # --- Initial Random Points ---
    print("\nGenerating initial random samples...")
    initial_params = opt.ask(n_initial_points)
    batch_objective(initial_params, desc="Initial Random Sampling")
    opt.tell(initial_params, [r['deaths_diff (vax_all True - False)'] for r in results_list])

    # --- Bayesian Iterations ---
    num_batches = (n_calls - n_initial_points) // parallel_batch_size
    for i in tqdm(range(num_batches), desc="Bayesian Optimization Progress"):
        next_params = opt.ask(parallel_batch_size)
        batch_results = batch_objective(next_params, desc=f"Batch {i+1}/{num_batches}")
        opt.tell(next_params, batch_results)

    # --- Save Results ---
    df_results = pd.DataFrame(results_list)
    df_results.to_csv('bayesian_optimization_results.csv', index=False)
    print("\nBayesian Optimization complete! Results saved to 'bayesian_optimization_results.csv'.")

    # --- Best Params ---
    best_idx = df_results['deaths_diff (vax_all True - False)'].idxmin()
    print("\nBest Parameters Found:")
    print(df_results.loc[best_idx])

    # --- Convergence Plot ---
    plt.figure(figsize=(10, 5))
    plot_convergence(opt.get_result())
    plt.show()

    # --- Extract Near-Optimal Region ---
    near_best_df = get_best_region(df_results, threshold_percent=threshold_percent)

    return df_results, near_best_df

# --- Run Everything ---
df_results, near_best_df = run_bayesian_optimization(
    n_calls=500,
    n_initial_points=50,
    parallel_batch_size=50,
    threshold_percent=10,
    num_workers=cpu_count()
)



--- Starting Bayesian Optimization ---

Generating initial random samples...


Initial Random Sampling:  32%|███▏      | 16/50 [04:29<11:13, 19.80s/it]

KeyboardInterrupt: 

# Understanding Gaussian Processes & Expected Improvement in Bayesian Optimization

---

## 📌 What is a Gaussian Process (GP)?

A **Gaussian Process (GP)** is a **probabilistic model** used to estimate unknown functions.

### 🚀 Intuition:

- Instead of fitting a single deterministic function, a GP models a **distribution over possible functions** that fit the observed data.
- It assumes:
  - **Any finite set of points** in the input space has a **joint multivariate Gaussian distribution**.

---

## 🟢 Key Components:

1. **Mean function:**
   - Denoted as:  
     $$ m(x) = \mathbb{E}[f(x)] $$
   - Represents the expected value of the function at each point.  
   - Often assumed to be zero for simplicity.

2. **Covariance function (Kernel):**
   - Denoted as:  
     $$ k(x, x') = \mathbb{E}[(f(x) - m(x))(f(x') - m(x'))] $$
   - Measures **similarity** between points.
   - Common kernels:
     - Radial Basis Function (RBF) / Squared Exponential.
     - Matern kernel.
   - Controls:
     - Smoothness.
     - Distance-based correlation.

---

## 📈 How GPs Are Used in Optimization:

Given some observed data points:

1. **GP regression** predicts:
   - **Mean prediction:**  
     $$ \mu(x) $$
   - **Variance prediction (uncertainty):**  
     $$ \sigma^2(x) $$

2. The optimizer uses both **$\mu(x)$** and **$\sigma(x)$** to decide where to evaluate next.

---

# 🟢 Expected Improvement (EI)

## 🚀 What is Expected Improvement?

- **EI is an acquisition function** used to balance:
  - **Exploitation** → sample where the predicted objective value is best.
  - **Exploration** → sample where the model is uncertain.

---

## 🔍 EI Formula:

Define:

- **$f^*$**: Current best observed value (e.g., lowest deaths difference so far).
- **$\mu(x)$**: Predicted mean at point $x$ (from GP).
- **$\sigma(x)$**: Predicted standard deviation at point $x$.

Then:

$$
EI(x) = \mathbb{E}[\max(0, f^* - f(x))]
$$

Closed-form formula:

$$
EI(x) = (f^* - \mu(x)) \cdot \Phi(Z) + \sigma(x) \cdot \phi(Z)
$$

Where:

- $$ Z = \frac{f^* - \mu(x)}{\sigma(x)} $$
- **$\Phi(Z)$**: Cumulative distribution function (CDF) of standard normal.
- **$\phi(Z)$**: Probability density function (PDF) of standard normal.

---

## 🟢 Intuition:

- **If $\mu(x)$ is much better than $f^*$** → EI is large → **Exploitation**.
- **If $\sigma(x)$ is large (high uncertainty)** → EI is large → **Exploration**.

Thus, **EI naturally balances exploration and exploitation!**

---

## 📊 Summary:

| Concept              | Meaning |
|---------------------|--------|
| **Gaussian Process** | Probabilistic model predicting both mean & uncertainty of objective function. |
| **Mean $\mu(x)$**    | Best guess of objective value at $x$. |
| **Variance $\sigma^2(x)$** | Uncertainty of the model at $x$. |
| **Expected Improvement (EI)** | Acquisition function deciding next sample by maximizing improvement expectation. |

---

## 🔥 Why It Works:

- **GPs provide a smooth, uncertainty-aware model.**
- **EI tells the optimizer where it's most beneficial to sample next:**
  - Either improve the best value.
  - Or reduce uncertainty in unknown areas.

---

## 🚀 Bonus:

Would you like to see a visualization of how a Gaussian Process and Expected Improvement evolve over iterations on a toy function?


# Adaptive Parameter Search (parallelizable)

In [ ]:
from multiprocessing import Pool, cpu_count
print(cpu_count())

8


In [ ]:
# --- Fixed Simulation Settings ---
SEED = 42
random.seed(SEED)

grid_size = 25
num_agents = 600
init_infected_proportion = 0.1
proportion_vulnerable = 0.1
infection_prob = 0.25
recovery_time = 30

base_sampling_ranges = {
    'Death Prob': (0.01, 0.1),
    'Vul Penalty': (0.2, 0.8),
    'Vax Effect': (0.5, 0.9),
    'Viral Age Effect': (0.05, 0.2),
    'Immune Adaptation Effect': (0.05, 0.2)
}

# --- Parameter Sampling ---
def sample_params(ranges):
    return {param: round(random.uniform(*rng), 3) for param, rng in ranges.items()}

# --- Worker Function: Single Simulation Task ---

def run_single_simulation(params):
    deaths = {}
    for vax_all in [True, False]:
        simulation = Simulation(
            grid_size=grid_size,
            num_agents=num_agents,
            agent_class=FullAgent,
            init_infected_proportion=init_infected_proportion,
            proportion_vulnerable=proportion_vulnerable,
            vul_penalty=params['Vul Penalty'],
            infection_prob=infection_prob,
            recovery_time=recovery_time,
            death_prob=params['Death Prob'],
            vax_vulnerable=True,
            vax_all=vax_all,
            vax_effect=params['Vax Effect'],
            viral_age_effect=params['Viral Age Effect'],
            immune_adaptation_effect=params['Immune Adaptation Effect'],
            plot=False
        )
        simulation.run(500)
        report = simulation.generate_simulation_report()
        deaths[vax_all] = report[0]

    # Check success condition
    if deaths[False] < deaths[True]:
        result = params.copy()
        result.update({
            'Deaths vax_all=False': deaths[False],
            'Deaths vax_all=True': deaths[True]
        })
        return result
    else:
        return None

# --- Parallel Search Function ---

def parallel_parameter_search(sampling_ranges, num_samples=500, stage_name='Stage', save_csv=False):
    print(f"\n{stage_name}: Running {num_samples} samples in parallel...")

    # Prepare parameters to sample
    param_list = [sample_params(sampling_ranges) for _ in range(num_samples)]

    # Use all available CPUs
    num_workers = cpu_count()
    print(f"Using {num_workers} CPU cores for parallelization.")

    with Pool(processes=num_workers) as pool:
        # Parallel map with progress bar
        results = list(tqdm.tqdm(pool.imap(run_single_simulation, param_list), total=num_samples))

    # Filter out None results
    successful_regions = [res for res in results if res is not None]

    df_success = pd.DataFrame(successful_regions)
    print(f"{stage_name}: Found {len(df_success)} successful regions.")

    if save_csv:
        df_success.to_csv(f'{stage_name.lower()}_successful_regions.csv', index=False)

    return df_success

# --- Analysis + Range Refinement ---

def analyze_and_refine_ranges(df_success, shrink_factor=1.0):
    refined_ranges = {}
    params = list(base_sampling_ranges.keys())
    stats = df_success[params].describe()

    print("\nRefining parameter ranges based on successful regions...")
    for param in params:
        mean = stats.loc['mean', param]
        std = stats.loc['std', param]
        param_min = max(stats.loc['min', param], mean - shrink_factor * std)
        param_max = min(stats.loc['max', param], mean + shrink_factor * std)
        refined_ranges[param] = (round(param_min, 3), round(param_max, 3))

        # Optional plot
        plt.figure(figsize=(8, 4))
        sns.histplot(df_success[param], bins=10, kde=True)
        plt.title(f'{param} Distribution (Successful Regions)')
        plt.show()

    print("New refined parameter ranges:", refined_ranges)
    return refined_ranges

# --- Recursive Refinement Pipeline ---

def recursive_parallel_search(initial_samples=1000, focused_samples=500, num_iterations=3, shrink_factor=1.0):
    print(f"\n--- Starting Recursive Parameter Search ({num_iterations} Iterations) ---\n")

    # Step 1: Initial Random Search
    df_initial = parallel_parameter_search(base_sampling_ranges, num_samples=initial_samples, stage_name='Initial', save_csv=True)
    if df_initial.empty:
        print("No successful regions found in initial search.")
        return

    current_ranges = analyze_and_refine_ranges(df_initial, shrink_factor=shrink_factor)

    # Step 2: Recursive Refinement
    all_results = [df_initial]
    for iteration in range(1, num_iterations + 1):
        print(f"\n--- Iteration {iteration} ---\n")
        df_refined = parallel_parameter_search(current_ranges, num_samples=focused_samples, stage_name=f'Focused_Iter{iteration}', save_csv=True)
        if df_refined.empty:
            print(f"No successful regions found in iteration {iteration}. Stopping early.")
            break
        all_results.append(df_refined)
        current_ranges = analyze_and_refine_ranges(df_refined, shrink_factor=shrink_factor)

    # Combine all results
    df_all = pd.concat(all_results, ignore_index=True)
    df_all.to_csv('all_successful_regions.csv', index=False)
    print("\nRecursive parameter search complete! All results saved.")

    return df_all

# --- Execute Full Pipeline ---

df_results = recursive_parallel_search(
    initial_samples=1000,
    focused_samples=500,
    num_iterations=3,
    shrink_factor=1.0
)



--- Starting Recursive Parameter Search (3 Iterations) ---


Initial: Running 1000 samples in parallel...
Using 8 CPU cores for parallelization.


 25%|██▌       | 253/1000 [05:01<15:05,  1.21s/it]